In [1]:
import pandas
import sklearn
import numpy as np

In [2]:
train = pandas.read_csv("task2_lemmas_train",names=["ID", "X", "y", "y1", "y2", "y3", "y4"], skiprows=1)
train = train
test = pandas.read_csv("task2_lemmas_test")
ans = pandas.read_csv("task2_lemmas_sample_submission")
additional = []
for index, row in train.iterrows():
    if isinstance(row.y1, str):
        additional.append((row.X, row.y1))
    if isinstance(row.y2, str):
        additional.append((row.X, row.y2))
    if isinstance(row.y3, str):
        additional.append((row.X, row.y3))
    if isinstance(row.y4, str):
        additional.append((row.X, row.y4))
for el in additional:
    train.loc[len(train)] = {"Id": None, "X": el[0], "y": el[1], "y1": None, "y2": None, "y3": None, "y4": None}
print train
train['tag'] = train.y.apply(lambda s: s[-1])
train.y = train.y.apply(lambda s: s[:-2])

          ID                  X                 y    y1    y2    y3    y4
0        1.0       vergognerete      vergognare+V   NaN   NaN   NaN   NaN
1        2.0       amnistiavate      amnistiare+V   NaN   NaN   NaN   NaN
2        3.0        menomazione     menomazione+N   NaN   NaN   NaN   NaN
3        4.0         sfaldavamo        sfaldare+V   NaN   NaN   NaN   NaN
4        5.0         sfodererei       sfoderare+V   NaN   NaN   NaN   NaN
5        6.0         ascondesti       ascondere+V   NaN   NaN   NaN   NaN
6        7.0      edifichereste       edificare+V   NaN   NaN   NaN   NaN
7        8.0         maschieran       maschiare+V   NaN   NaN   NaN   NaN
8        9.0      transennasser     transennare+V   NaN   NaN   NaN   NaN
9       10.0         computando       computare+V   NaN   NaN   NaN   NaN
10      11.0         accudisser        accudire+V   NaN   NaN   NaN   NaN
11      12.0       diromperanno       dirompere+V   NaN   NaN   NaN   NaN
12      13.0      intercollegai  inter

In [3]:
from sklearn.linear_model import LogisticRegression

In [4]:
from sklearn.model_selection import cross_val_score
from collections import defaultdict

In [5]:
class ExtractFeatures(sklearn.base.BaseEstimator, sklearn.base.TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return [self.extract(word) for word in X]

    def extract(self, word):
        features = defaultdict(lambda: 0)
        if word[0].isupper() and not word.isupper():
            features['capitalized'] = 1
        if word.isupper():
            features['allcaps'] = 1
        word = word.lower()
        for i in range(min(len(word), 7)):
            features["pref" + word[:i]] += 1
            features["suf" + word[-i:]] += 1
            if i > 0:
                features["word" + word[:-i] + "*"*i] += 1
        for length in range(1, 6):
            for i in range(len(word) - length + 1):
                features[word[i:i + length]] += 1
            for offset in range(1, 4):
                if len(word) - offset - length < 0:
                    continue
                features[word[len(word) - offset - length: len(word) - offset] + "*"*offset] += 1
        features["length"] = len(word)
        features["rect_len"] = max(len(word) - 10, 0)
        return features
        

In [6]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import make_pipeline

In [7]:
train.X = [unicode(x, "utf-8") for x in train.X]
test.X = [unicode(x, "utf-8") for x in test.X]
train.y = [unicode(x, "utf-8") for x in train.y]

In [8]:
vectorizer = DictVectorizer()
train_data = vectorizer.fit_transform(ExtractFeatures().transform(train.X))

In [9]:
est = make_pipeline(sklearn.preprocessing.MaxAbsScaler(), LogisticRegression(class_weight='balanced'))
est.fit(train_data, train.tag)
#sklearn.model_selection.cross_val_score(est, train_data, train.tag)

Pipeline(steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('logisticregression', LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False))])

In [10]:
def longest_common_substring(s1, s2):
    m = [[0] * (1 + len(s2)) for i in xrange(1 + len(s1))]
    longest, x_longest, y_longest = 0, 0, 0
    for x in xrange(1, 1 + len(s1)):
        for y in xrange(1, 1 + len(s2)):
            if s1[x - 1] == s2[y - 1]:
                m[x][y] = m[x - 1][y - 1] + 1
                if m[x][y] > longest:
                    longest = m[x][y]
                    x_longest = x
                    y_longest = y
            else:
                m[x][y] = 0
    return x_longest, y_longest, longest


class Node(object):
    def __hash__(self):
        return unicode(self).__hash__()
    
    def __eq__(self, other):
        return unicode(self) == unicode(other)
    
    def __repr__(self):
        return self.__unicode__()

class SplitNode(Node):
    def __init__(self, bounds, left, right):
        self.bounds = bounds
        self.left = left
        self.right = right

    def convert(self, word):
        if self.bounds[1] > len(word):
            return None
        left = self.left.convert(word[:self.bounds[0]])
        if left is None:
            return None
        right = self.right.convert(word[self.bounds[1]:])
        if right is None:
            return None
        return "".join([left, word[self.bounds[0]:self.bounds[1]], right])
    
    def alignment(self, word):
        al = self.left.alignment(word[:self.bounds[0]]) + self.right.alignment(word[self.bounds[1]:])
        if self.bounds[1] - self.bounds[0] > 3:
            al.append(u"R{0}".format(word[self.bounds[0]:self.bounds[1]]))
        return al

    def __unicode__(self):
        return u"{}<{}><{}>".format(self.bounds, self.left, self.right)

class ReplaceNode(Node):
    def __init__(self, pattern, target):
        self.pattern = pattern
        self.target = target

    def convert(self, word):
        if word == self.pattern:
            return self.target
        else:
            return None
    
    def alignment(self, word):
        if len(self.pattern) == 0 and len(self.target) == 0:
            return []
        return [unicode(self)]

    def __unicode__(self):
        return u"{}/{}".format(self.pattern, self.target)


def make_tree(word, lemma):
    x_longest, y_longest, longest = longest_common_substring(word, lemma)
    if longest <= 2:
        return ReplaceNode(word, lemma)
    else:
        left = make_tree(word[:x_longest - longest], lemma[:y_longest - longest])
        right = make_tree(word[x_longest:], lemma[y_longest:])
        bounds = (x_longest - longest, x_longest)
        return SplitNode(bounds, left, right)


In [11]:
from collections import defaultdict
from sklearn.feature_extraction import FeatureHasher

In [12]:
hasher = FeatureHasher()

In [13]:
tree_count = defaultdict(lambda: 0)
for word, lemma in zip(train.X, train.y):
    tree_count[make_tree(word, lemma)] += 1

In [14]:
trees = [k for k, v in tree_count.iteritems() if v > 0]
print len(trees)

3637


In [15]:
def add_feature(features, feature, tag=None):
    features[feature] = 1
    if tag is not None:
        features[tag+feature] = 1

def get_features(word, lemma, tag, tree=None):
    features = defaultdict(lambda: 0)
    if tree is None:
        tree = make_tree(word, lemma)
    add_feature(features, unicode(tree), tag)
    for el in tree.alignment(word):
        add_feature(features, el, tag)
        add_feature(features, u"{}:{}".format(el, tree), tag)
        if el[0] == 'R':
            add_feature(features, u"{}->{}".format(el, lemma), tag)
    add_feature(features, lemma, tag)
    for i in range(1, min(len(lemma), 5)):
        #add_feature(features, "pref" + lemma[:i], tag)
        add_feature(features, "suf" + lemma[-i:], tag)
    features["tag_score"] = est.predict_proba(vectorizer.transform(ExtractFeatures().extract(word)))[0, list(est.classes_).index(tag)]
    return features

In [16]:
X_train = train[:80000]
X_test = train[80000:]

In [17]:
y = []
def get_data():
    for index, row in train.iterrows():
        if index % 100 == 0:
            print '.',
        if index % 1000 == 0:
            print index
        word = row.X
        has_true = False
        skip = 1
        for i in range(index % skip, len(trees), skip):
            tree = trees[i]
            lemma = tree.convert(word)
            if lemma is None:
                continue
            for tag in est.classes_:
                true = tag == row.tag and lemma == row.y
                if true:
                    has_true = True
                y.append(int(true))
                yield get_features(word, lemma, tag, tree)
        if not has_true:
            yield get_features(word, row.y, row.tag)
            y.append(1)
X = hasher.transform(get_data())

. 0
. . . . . . . . . . 1000
. . . . . . . . . . 2000
. . . . . . . . . . 3000
. . . . . . . . . . 4000
. . . . . . . . . . 5000
. . . . . . . . . . 6000
. . . . . . . . . . 7000
. . . . . . . . . . 8000
. . . . . . . . . . 9000
. . . . . . . . . . 10000
. . . . . . . . . . 11000
. . . . . . . . . . 12000
. . . . . . . . . . 13000
. . . . . . . . . . 14000
. . . . . . . . . . 15000
. . . . . . . . . . 16000
. . . . . . . . . . 17000
. . . . . . . . . . 18000
. . . . . . . . . . 19000
. . . . . . . . . . 20000
. . . . . . . . . . 21000
. . . . . . . . . . 22000
. . . . . . . . . . 23000
. . . . . . . . . . 24000
. . . . . . . . . . 25000
. . . . . . . . . . 26000
. . . . . . . . . . 27000
. . . . . . . . . . 28000
. . . . . . . . . . 29000
. . . . . . . . . . 30000
. . . . . . . . . . 31000
. . . . . . . . . . 32000
. . . . . . . . . . 33000
. . . . . . . . . . 34000
. . . . . . . . . . 35000
. . . . . . . . . . 36000
. . . . . . . . . . 37000
. . . . . . . . . . 38000
. . . . . . . . .

In [18]:
model = LogisticRegression(solver='liblinear')
model.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [19]:
answers = []
for index, row in test.iterrows():
    if index % 100 == 0:
        print '.',
    if index % 1000 == 0:
        print index
    word = row.X
    best_lemma = 'test'
    best_p = 0
    best_tag = 'N'
    for tree in trees:
        lemma = tree.convert(word)
        if lemma is None:
            continue
        for tag in est.classes_:
            p = model.predict_proba(hasher.transform([get_features(word, lemma, tag, tree)]))[0, 1]
            if p > best_p:
                best_p = p
                best_tag = tag
                best_lemma = lemma
    answers.append(u"{}+{}".format(best_lemma, best_tag))

. 0
. . . . . . . . . . 1000
. . . . . . . . . . 2000
. . . . . . . . . . 3000
. . . . . . . . . . 4000
. . . . . . . . . . 5000
. . . . . . . . . . 6000
. . . . . . . . . . 7000
. . . . . . . . . . 8000
. . . . . . . . . . 9000
. . . . . . . . . . 10000
. . . . . . . . . . 11000
. . . . . . . . . . 12000
. . . . . . . . . . 13000
. . . . . . . . . . 14000
. . . . . . . . . . 15000
. . . . . . . . . . 16000
. . . . . . . . . . 17000
. . . . . . . . . . 18000
. . . . . . . . . . 19000
. . . . . . . . . . 20000
. . . . . . . . . . 21000
. . . . . . . . . . 22000
. . . . . . . . . . 23000
. . . . . . . . . . 24000
. . . . . . . . . . 25000
. . . . . . . . . . 26000
. . . . . . . . . . 27000
. . . . . . . . . . 28000
. . . . . . . . . . 29000
. . . . . .


In [20]:
ans.Category = answers
ans.to_csv("lemma.csv", index=False, encoding="utf-8")